In [1]:
from pygments.formatters.html import webify
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import sqlite3

In [2]:
# 0. Tạo cơ sở dữ liệu
conn = sqlite3.connect('musician.db')
c = conn.cursor()
try:
    c.execute('''
        CREATE TABLE musicians (
            id INTEGER primary key autoincrement,
            name_of_the_band TEXT,
            year_active INTEGER
        )
    ''')
except Exception as e:
    print(e)



In [3]:
def them(name_of_the_band, year_active):
    conn = sqlite3.connect('musician.db')
    c = conn.cursor()
    # Them vao co so du lieu
    c.execute('''
        INSERT INTO musician(name_of_the_band, year_active)
        VALUES (:name_of_the_band, :year_active)
    ''',
      {
          'name_of_the_band': name_of_the_band,
          'year_active': year_active
      })
    conn.commit()
    conn.close()

In [4]:
#tạo dataframe rỗng và dictionnary
all_links = []
musician_links = []


In [5]:
# # Đường dẫn đến file thực thi geckodriver
# gecko_path = r"C:\Users\Admin\OneDrive\Tài liệu\GitHub\Ma_nguon_mo_khoa\BTVN-musician\geckodriver.exe"

# # Khởi tởi đối tượng dịch vụ với đường geckodriver
# ser = Service(gecko_path)

# # Tạo tùy chọn
# options = webdriver.Chrome.options.Options()
# options.binary_location ="C:/Program Files/Mozilla Firefox/firefox.exe"
# # Thiết lập firefox chỉ hiện thị giao diện
# options.headless = False


In [6]:
# Tạo op để chạy chế độ ẩn 
chrome_options = Options()
chrome_options.add_argument("--headless=new")  
chrome_options.add_argument("--disable-gpu")  
chrome_options.add_argument("--no-sandbox")  
chrome_options.add_argument("--disable-dev-shm-usage") 
chrome_options.add_argument("--log-level=3")  
chrome_options.add_argument("--window-size=1920x1080")  

In [7]:
#truy cập vô trang web 
driver = webdriver.Chrome(chrome_options)
url = 'https://en.wikipedia.org/wiki/Lists_of_musicians#A'

#mở tràn web
driver.get(url)

#dừng khoảng 2s
time.sleep(2)

try:
    #lấy tất cả các thẻ ul trong web danh mục
    ul_tags = driver.find_elements(By.TAG_NAME, "ul")
    print(len(ul_tags))

    #chọn ul thứ 22
    ul_musican = ul_tags[21]
    #lấy tất cả link chứa thông tin nhạc sĩ bắt đầu bằng chữ A thuộc ul_musican
    li_tags = ul_musican.find_elements(By.TAG_NAME, "li")
    print(len(li_tags))

    # tạo danh sách các url
    links = [tag.find_element(By.TAG_NAME,"a").get_attribute("href") for tag in li_tags]
    for x in links:
        all_links.append(x)
except:
    print("Error!!!!")
#tat man hinh
driver.quit()

141
10


In [8]:
#kiểm tra all_links có dữ liệu chưa
print(all_links)

['https://en.wikipedia.org/wiki/List_of_acid_rock_artists', 'https://en.wikipedia.org/wiki/List_of_adult_alternative_artists', 'https://en.wikipedia.org/wiki/List_of_alternative_country_musicians', 'https://en.wikipedia.org/wiki/List_of_alternative_hip_hop_artists', 'https://en.wikipedia.org/wiki/List_of_alternative_metal_artists', 'https://en.wikipedia.org/wiki/List_of_alternative_rock_artists', 'https://en.wikipedia.org/wiki/List_of_ambient_music_artists', 'https://en.wikipedia.org/wiki/List_of_anarcho-punk_bands', 'https://en.wikipedia.org/wiki/List_of_Arabic_pop_musicians', 'https://en.wikipedia.org/wiki/List_of_avant-garde_metal_artists']


In [9]:
#truy cập vô đường link đầu tiên của all_links
artists_driver = webdriver.Chrome(chrome_options)
artists_driver.get(all_links[0])

#dừng khoảng 2s
time.sleep(2)

try:
     #lấy tất cả các the ul của list of acid rock artists
    ul_artists_tags = artists_driver.find_elements(By.TAG_NAME, "ul")
    print(len(ul_artists_tags))

    #chọn ul thứ 25
    ul_artist = ul_artists_tags[24]
    #lấy tất cả link chứa thông tin thuộc artists
    li_artist = ul_artist.find_elements(By.TAG_NAME, "li")
    print(len(li_artist))

    # tạo danh sách các url của artist
    links_artist = [artist_tag.find_element(By.TAG_NAME,"a").get_attribute("href") for artist_tag in li_artist]
    for x in links_artist:
        musician_links.append(x)
except:
    print("Error!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
artists_driver.quit()

48
40


In [14]:
#kiểm tra musician có dữ liệu chưa
print(musician_links)

['https://en.wikipedia.org/wiki/The_13th_Floor_Elevators', 'https://en.wikipedia.org/wiki/Alice_Cooper_(band)', 'https://en.wikipedia.org/wiki/The_Amboy_Dukes_(band)', 'https://en.wikipedia.org/wiki/Amon_D%C3%BC%C3%BCl', 'https://en.wikipedia.org/wiki/Big_Brother_and_the_Holding_Company', 'https://en.wikipedia.org/wiki/Black_Sabbath', 'https://en.wikipedia.org/wiki/Blue_Cheer', 'https://en.wikipedia.org/wiki/Blues_Magoos', 'https://en.wikipedia.org/wiki/The_Charlatans_(American_band)', 'https://en.wikipedia.org/wiki/Count_Five', 'https://en.wikipedia.org/wiki/Country_Joe_and_the_Fish', 'https://en.wikipedia.org/wiki/Coven_(band)', 'https://en.wikipedia.org/wiki/Cream_(band)', 'https://en.wikipedia.org/wiki/Deep_Purple', 'https://en.wikipedia.org/wiki/The_Deviants_(band)', 'https://en.wikipedia.org/wiki/The_Doors', 'https://en.wikipedia.org/wiki/The_Electric_Prunes', 'https://en.wikipedia.org/wiki/The_Fugs', 'https://en.wikipedia.org/wiki/Grateful_Dead', 'https://en.wikipedia.org/wiki/T

In [13]:
#lấy thông tin của các nhạc sĩ ca sĩ
count = 0
for link in musician_links:
    if(count >= 2):
        break
    count += 1
    print(link)
    try:
        #khởi tạo webdriver
        driver = webdriver.Chrome()
        #mở trang web
        url = link
        driver.get(url)
        #đợi khoảng 2s
        time.sleep(2)
        #lấy tên ban nhạc
        try:
            name_band = driver.find_element(By.TAG_NAME, "h1").text
        except:
            name_band = ""

        #lay năm hoat dong
        try:
            year_active_element = driver.find_element(By.XPATH, value='//span[contains(text(),"Years active")]/parent::*/following-sibling::td')
            year_active = year_active_element.text
            
        except:
            year_active = ""

        # #tạo dictionanty để thêm thông tin nhạc sĩ
        # musician = {'name of the band': name_band, 'years active': year_active}
        # #chuyển đổi dictionary thành dataframe
        # musician_df = pd.DataFrame([musician])
        # #thêm thông tin vào df chính
        # df = pd.concat([df, musician_df], ignore_index=True)

        them(name_band, year_active)
        #đóng web
        driver.quit()
    except:
        print("Error!!!!!!!!!!!!!!!")

https://en.wikipedia.org/wiki/The_13th_Floor_Elevators
Error!!!!!!!!!!!!!!!
https://en.wikipedia.org/wiki/Alice_Cooper_(band)
Error!!!!!!!!!!!!!!!
